<a href="https://colab.research.google.com/github/DasagociCorpus/Pruebas/blob/main/Profesional_III_(2400015V)_Prueba_Redes_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **a. Supuestos**

**Independencia de Observaciones:** Se asume que cada transacción es independiente de las demás por lo cual no existirián dependencias ocultas que puedan sesgar los resultados.

**Representatividad del Submuestreo:** En caso de que el grafo contenga más de 1000 nodos, se asume que el submuestreo aleatorio de 1000 nodos mantiene la representatividad y características estructurales del grafo completo.

**Detección de Comunidades:** Se asume que el algoritmo de Louvain es adecuado para detectar comunidades significativas dentro del grafo.

**Homogeneidad de Atributos:** Se supone que los valores en la columna 'MONTO' son comparables entre sí.


# **b. Marco Teórico**
**Análisis de Grafos:** Los grafos son una representación visual de una red o sistema complejo. Un grafo se compone de  nodos (vértices) y aristas (bordes), que se pueden representar de diferentes maneras. En la teoría de grafos, los nodos se representan con puntos y las aristas con líneas que conectan los nodos. Los grafos se utilizan para representar relaciones entre objetos, como por ejemplo en una red social, donde los nodos pueden representar usuarios y las aristas las conexiones entre ellos. (https://cienciaconjunta.com/matematicas/teoria-de-grafos-conceptos-basicos-y-aplicaciones-en-redes-y-sistemas-complejos/)

**Algoritmo de Louvain:** Los algoritmos de detección de comunidades son esenciales en el análisis de datos, especialmente en el contexto de grafos. Permiten identificar agrupaciones naturales de nodos altamente interconectados en una red, lo que facilita la comprensión de las relaciones y estructuras dentro de los datos. En el caso del algoritmo de Louvain, éste se encuentra basado en la modularidad de los datos. Este realiza una evaluación del conjunto de datos y compara la densidad de aristas que están presentes dentro o fuera de la comunidad. Al optimizar este valor de iteración se obtiene un estimado de agrupación de los nodos que pertenecen a una red. (https://www.grapheverywhere.com/algoritmo-de-louvain/)

**Centralidad en Grafos:**La centralidad mide la importancia de los nodos dentro de un grafo. Las métricas comunes incluyen:

* Grado (Degree): Número de aristas conectadas a un nodo.
* PageRank: Mide la importancia de los nodos basándose en la estructura de los enlaces entrantes.
* Centralidad de Eigenvector: Evalúa la influencia de un nodo basada en la influencia de sus vecinos.

# **c. Exploración y Tratamiento de Datos**

## **Importar bibliotecas**

Importamos las bibliotecas que necesitamos para el análisis de datos y la visualización. Esto incluye pandas para la manipulación de datos, matplotlib para la visualización y networkx para el análisis de grafos.

In [ ]:
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import networkx as nx
import networkx.algorithms.community as nx_comm

warnings.filterwarnings("ignore")

## **Cargar los datos:**

Leemos los datos desde una URL utilizando pd.read_csv de pandas. Convertimos la columna 'MONTO' a tipo float para poder realizar cálculos numéricos con ella.

In [ ]:
data = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQV_Yltirr7KzrZJuuDw02xW7rPp_8vlyU1cSKh85Z_fFweg2k7L4uH-VFgsvZm9XtwGUMDzyqauJVk/pub?gid=2130478625&single=true&output=csv'
df = pd.read_csv(data, dtype=str)
df['MONTO'] = df['MONTO'].astype(float)

## **Submuestreo de nodos y creación del grafo:**

Si el número de nodos en el grafo es mayor que 1000, submuestreamos aleatoriamente 1000 nodos para reducir el tamaño del grafo y mejorar la eficiencia del procesamiento.

A continuación, creamos un grafo utilizando nx.from_pandas_edgelist, que toma un df y crea un grafo con networkx basado en las columnas de origen, destino y atributos del DataFrame, en este caso Source, Target y MONTO. (https://networkx.org/)

In [ ]:
# Submuestreo de nodos
if len(df) > 1000:
    df = df.sample(n=1000, random_state=42)

#creación del grafo
G = nx.from_pandas_edgelist(df, 'Source', 'Target', edge_attr='MONTO')


## **Detectar comunidades con Louvain:**

Utilizamos el algoritmo de Louvain para detectar comunidades en el grafo. Esto nos permite identificar grupos de nodos altamente conectados.

Es importante recalcar que el algoritmo de Louvain



> [...] tiene como objetivo optimizar un concepto matemático denominado modularidad, [el cual es] una medida que se utiliza en la teoría de grafos para evaluar la calidad de una partición.

De una forma más precisa:

>La modularidad se define como la diferencia entre el número de enlaces observado dentro de las comunidades y el número de enlaces esperados por azar. Una alta modularidad indica que las comunidades encontradas tienen más conexiones que las esperadas por azar y, por tanto, son compactas y bien definidas.

(Fuente: https://cienciadedatos.net/documentos/pygml02-detecion-comunidades-grafos-redes-python)






In [ ]:
comunidades = list(nx_comm.louvain_communities(G, weight='weight', resolution=0.2))

## **Visualizar el grafo:**

Visualizamos el grafo utilizando nx.draw. Establecemos el diseño del grafo con nx.spring_layout y asignamos colores a los nodos según las comunidades detectadas.

In [ ]:
color_map = [i % 10 for i, _ in enumerate(G.nodes())]
plt.figure(figsize=(9, 5))
pos = nx.spring_layout(G)
nx.draw(G, pos, node_color=color_map, cmap=plt.cm.tab10, with_labels=False, node_size=15)
plt.show()

## **Calcular métricas de centralidad:**

Calculamos varias métricas de centralidad para los nodos del grafo, incluyendo

degree: número de aristas adyacentes al nodo. El grado de nodo ponderado es la suma de los pesos de los bordes incidentes en ese nodo. Este objeto proporciona un iterador para (nodo, grado), así como una búsqueda del grado de un solo nodo. (https://networkx.org/documentation/stable/reference/classes/generated/networkx.Graph.degree.html)

PageRank: calcula una clasificación de los nodos en el gráfico G en función de la estructura de los enlaces entrantes. (https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.link_analysis.pagerank_alg.pagerank.html)

centralidad de eigenvector: calcula la centralidad de un nodo sumando la centralidad de sus predecesores. (https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.centrality.eigenvector_centrality.html)

In [ ]:
centralidad = pd.DataFrame({
    'Source': dict(nx.degree(G)),
    'Target': nx.eigenvector_centrality(G),
    'MONTO': nx.pagerank(G)
}).sort_index()

## **Mostrar los principales nodos para cada métrica:**

Imprimimos los 5 nodos principales para cada métrica de centralidad calculada.

In [ ]:
for col in centralidad.columns:
    top_nodos = centralidad[col].nlargest(5).index.tolist()
    top_nodos.sort()
    print(f"Nodos con mayor {col}: {top_nodos}")

## **Aplicación en busqueda:**

Solicitamos al usuario que elija entre 'Source' o 'Target' y que ingrese el valor del nodo que desea visualizar.

Filtramos el DataFrame para obtener las aristas que involucran al nodo seleccionado por el usuario. Creamos un subgrafo utilizando nx.from_pandas_edgelist y visualizamos este subgrafo utilizando nx.draw. de esta forma podríamos observar las redes transaccionales desde un origen o destino.

In [ ]:
opcion = input("¿Qué quieres visualizar, 'Source' o 'Target'? ").strip().lower()
while opcion not in ['source', 'target']:
    print("Opción no válida. Por favor, ingresa 'Source' o 'Target'.")
    opcion = input("¿Qué quieres visualizar, 'Source' o 'Target'? ").strip().lower()

nodo = input(f"Por favor, ingresa el valor del nodo {opcion.capitalize()}: ")
if nodo in df[opcion.capitalize()].unique():
    df_filtrado = df[(df['Source' if opcion == 'source' else 'Target'] == nodo)]
    G_nodo = nx.from_pandas_edgelist(df_filtrado, 'Source', 'Target')
    plt.figure()
    pos = nx.spring_layout(G_nodo)
    nx.draw(G_nodo, pos, with_labels=True, node_size=300, node_color='skyblue', edge_color='gray', font_size=10)
    plt.title(f'Grafo del nodo {nodo} (basado en {opcion.capitalize()})')
    plt.show()
else:
    print(f"El nodo {nodo} no existe en la columna {opcion.capitalize()}.")

# **d. Modelos o Metodologías Utilizadas**

Algoritmo de Louvain, Visualización del Grafo y Métricas de Centralidad.

# **e. Elección del Mejor Modelo/Metodología y los Criterios de su Selección**

Para el presente ejercicio se tuvo en cuenta en primera instancia la interpretabilidad y visualización que, mendiante nx.draw, ilustra claramente de la estructura del grafo y la distribución de comunidades.

Por otra parte, la modularidad Alta desde el algoritmo de Louvain para la detección de comunidades.


# **f. Conclusiones y Recomendaciones para el Trabajo Futuro**

**Conclusiones:**

**Visualización Efectiva:** La visualización del grafo permitió una comprensión clara de la estructura de la red y la distribución de las comunidades.

**Métricas de Centralidad:** Las métricas de centralidad calculadas proporcionaron información efectiva sobre los nodos más importantes e influyentes en la red.

---

**Recomendaciones para el Trabajo Futuro:**

**Ampliar el Análisis de Datos:** Incluir más atributos y características en el análisis para obtener una comprensión más profunda de las relaciones y patrones en la red.

**Explorar Otros Algoritmos de Comunidad:** Validar algoritmos como el de Girvan-Newman o Infomap que nos permita debatir sobre laa comparar y validar los resultados.

**Optimización de Parámetros:** Ajustar los parámetros del algoritmo de Louvainpara explorar cómo afectan a la detección de comunidades.

**Inclusión de Análisis Temporal:** Validación de posibles insumos temporales que permitan realizar un análisis dinámico para estudiar cómo evolucionan las comunidades y las métricas de centralidad a lo largo del tiempo.

**Automatización y Escalabilidad:** Desarrollar un pipeline automatizado para el análisis y visualización del grafo, que pueda manejar grandes volúmenes de datos de manera eficiente.


## **Información de Sistema**

In [ ]:
!pip install sinfo
from sinfo import sinfo
sinfo()

In [ ]:
!jupyter nbconvert --to html "/content/Profesional_III_(2400015V)_Prueba_Redes_Final.ipynb"